In [44]:
from qiskit_ibm_runtime.fake_provider import FakeTorino
from qiskit import QuantumCircuit

import time
import numpy as np

In [45]:
#Define custom qiskit backend of a line of qubits
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler import CouplingMap

num_qubits_backend = 8
# coupling_map = CouplingMap.from_line(num_qubits_backend)
#Ladder of qubits
couplinglist = []
for i in range(0,num_qubits_backend-1, 2):
    if i+2 < num_qubits_backend:
        couplinglist.append([i,i+2])
    if i+3 < num_qubits_backend:
        couplinglist.append([i+1,i+3])
    if i+1 < num_qubits_backend:
        couplinglist.append([i,i+1])

coupling_map = CouplingMap(couplinglist=couplinglist)

line_backend = GenericBackendV2(num_qubits=num_qubits_backend, coupling_map=coupling_map, basis_gates=['cx', 'rz', 'rx', 'ry', 'h'])

In [46]:
print(coupling_map)

[[0, 2], [1, 3], [0, 1], [2, 4], [3, 5], [2, 3], [4, 6], [5, 7], [4, 5], [6, 7]]


In [47]:
#QFT circuit
num_qubits = 8
qc = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc.h(i)
    for j in range(i+1, num_qubits):
        qc.cp(np.pi/(2**(j-i)), j, i)

In [48]:
# qc.draw()

Default transpiler

In [49]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=line_backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

# default_qc.draw()

Time taken:  0.5353250503540039
OrderedDict({'rz': 127, 'cx': 84, 'rx': 39, 'ry': 34, 'h': 27})
Number of 2-qubit gates:  84
Number of 1-qubit gates:  227


Transpiler with custom options

In [50]:
# custom_pm = generate_preset_pass_manager(backend=line_backend, optimization_level=3, layout_method='dense')#, routing_method='sabre')

# t1 = time.time()
# custom_qc = custom_pm.run(qc)
# t2 = time.time()
# print("Time taken: ", t2-t1)

# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)
# print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
# print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

In [51]:
# #Convert qc to DAG
# from qiskit.converters import circuit_to_dag
# dag = circuit_to_dag(qc)

# #Visualize DAG
# from qiskit.visualization import dag_drawer
# dag_drawer(dag)

In [52]:
#Test with UCC transpiler
import sys
sys.path.append('../')

import ucc.transpilers.ucc_defaults as ucc_defaults

ucc_transpiler = ucc_defaults.UCCDefault1()
t1 = time.time()
ucc_qc = ucc_transpiler.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = ucc_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.14137697219848633
OrderedDict({'rz': 92, 'cx': 56, 'ry': 8})
Number of 2-qubit gates:  56
Number of 1-qubit gates:  100


In [53]:
print(ucc_qc)

global phase: 4.7185
      ┌──────────┐┌────────┐┌───┐┌──────────┐┌───┐┌─────────┐           ┌───┐»
q_0: ─┤ Ry(-π/2) ├┤ Rz(-π) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├───────────┤ X ├»
      ├─────────┬┘└────────┘└─┬─┘└──────────┘└─┬─┘├─────────┴┐┌────────┐└─┬─┘»
q_1: ─┤ Rz(π/4) ├─────────────■────────────────■──┤ Ry(-π/2) ├┤ Rz(-π) ├──┼──»
      ├─────────┤                                 └──────────┘└────────┘  │  »
q_2: ─┤ Rz(π/8) ├─────────────────────────────────────────────────────────■──»
      ├─────────┴┐                                                           »
q_3: ─┤ Rz(π/16) ├───────────────────────────────────────────────────────────»
      ├──────────┤                                                           »
q_4: ─┤ Rz(π/32) ├───────────────────────────────────────────────────────────»
      ├──────────┤                                                           »
q_5: ─┤ Rz(π/64) ├───────────────────────────────────────────────────────────»
     ┌┴──────────┤             